<a href="https://colab.research.google.com/github/lizehrhardt/recaptcha/blob/main/captcha_vgg_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model 

In [ ]:
large_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Mountain', 'Other', 'Palm', 'Stairs', 'Traffic Light']
image_size = 224
batch_size = 20

large_train_path = "/content/drive/MyDrive/captcha/large/train"
large_test_path = "/content/drive/MyDrive/captcha/large/val"

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory=large_train_path, target_size=(image_size, image_size))

validation_datagen = keras.preprocessing.image.ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(directory=large_test_path, target_size=(image_size, image_size))

Found 16439 images belonging to 13 classes.
Found 4117 images belonging to 13 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 3648s 37s/step - loss: 1.9253 - acc: 0.8321 - val_loss: 0.6161 - val_acc: 0.8711
Epoch 2/10
100/100 [==============================] - 3617s 36s/step - loss: 0.5027 - acc: 0.8697 - val_loss: 0.3179 - val_acc: 0.9181
Epoch 3/10
100/100 [==============================] - 3612s 36s/step - loss: 0.3284 - acc: 0.9122 - val_loss: 0.2831 - val_acc: 0.9212
Epoch 4/10
100/100 [==============================] - 3617s 36s/step - loss: 0.3041 - acc: 0.9183 - val_loss: 0.2791 - val_acc: 0.9227
Epoch 5/10
100/100 [==============================] - 3609s 36s/step - loss: 0.2956 - acc: 0.9212 - val_loss: 0.2781 - val_acc: 0.9229
Epoch 6/10
100/100 [==============================] - 3620s 36s/step - loss: 0.2916 - acc: 0.9223 - val_loss: 0.2768 - val_acc: 0.9229
Epoch 7/10
100/100 [==============================] - 3614s 36s/step - loss: 0.2935 - acc: 0.9212 - val_loss: 0.2759 - val_acc: 0.9231
Epoch 8/10
100/100 [==============================] - 3

In [ ]:
model.save('VGG_CaptchaModel_v1.h5')